## Benvenuto, iniziamo!

Questo è un **Notebook in Python**, cioè uno strumento interattivo per scrivere ed eseguire del codice in modo dinamico. È composto da *celle* che possono contenere testo (come questa) oppure codice Python (come la prossima).

Per *eseguire* i comandi in una cella, puoi:
- cliccare sul simbolo "Play" a sinistra della cella, oppure
- selezionare la cella e premere Shift + Invio sulla tastiera

Se non è già stato selezionato, al primo avvio Visual Studio potrebbe chiederti con quale *kernel* eseguire il codice (in pratica... con quale "versione" di Python): tu seleziona *Python Environments* e poi *env*. Abbiamo configurato noi l'ambiente così da farti trovare tutte le librerie necessarie già pronte per l'uso!

Perché tutto funzioni come previsto, ricordati di eseguire ogni cella di codice prima di passare alla successiva. Le variabili, funzioni e classi definite in una cella sono utilizzabili in tutto il notebook, una volta che la cella viene eseguita.

Per il resto, è il solito Python. Buon lavoro! 🚀


**Prova a eseguire la prossima cella:**

In [ ]:
# Un semplice test per verificare che funzioni...
print("Hai eseguito la cella!")

La cella ha stampato una riga di testo. In generale però si vede che la cella è stata eseguita anche dalla spunta verde che compare al suo fianco. Più avanti può esserti comodo controllare quali celle hai già eseguito osservando la spunta verde.

Per ora ti consigliamo di non modificare quel che trovi nelle celle di codice, ma **eseguirle soltanto**, cercando di capire che succede.

Poi potrai tornare indietro e modificarne alcuni pezzi per vedere come cambiano i risultati. Ti indicheremo nel testo dove potrebbe essere interessante modificare qualcosa.

È importante **eseguire le celle in ordine**, senza saltarne nessuna, altrimenti il computer si lamenterà e darà dei messaggi di errore. Se succede, non ti preoccupare: basterà andare a ricontrollare cosa hai saltato, eseguirlo e poi eseguire nuovamente anche la cella che ha dato errore. Non c'è niente che puoi davvero rompere qui dentro. E se hai bisogno di aiuto, chiamaci!

# Il problema: aspettativa di vita nei paesi del mondo

Immagina di essere parte dell'Organizzazione Mondiale della Sanità e di voler capire quali sono i fattori che influenzano l'aspettativa di vita (*life expectancy*) nelle varie nazioni.

Che il Prodotto Interno Lordo (PIL o GDP in inglese) del paese conti qualcosa? O che sia rilevante il numero medio di anni di istruzione? O che sia importante principalmente quanto sono presenti malattie come AIDS/HIV, epatite e difterite? O magari sarà il tasso di sottopeso nei bambini tra i 5 e i 9 anni l'indicatore più importante per prevedere l'aspettativa di vita lunga o corta?

Più probabilmente sarà un insieme di fattori a determinare la probabilità che le persone nate in un certo Paese vivano più o meno a lungo...

# Parte noiosa: procurarsi i dati!
Spesso questa è una delle parti più difficili della Data Science. Per fortuna, alcuni nostri colleghi hanno già raccolto dati per molti Paesi e li hanno inseriti in una tabella. Quindi noi partiremo con dei dati già pronti, dobbiamo solo caricarli in Python: esegui la prossima cella per continuare!

Se sei curioso, la fonte dei dati è https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who/data. 

In [ ]:
# Qui diciamo al codice di importare alcune librerie che poi useremo
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
import os

import random
random.seed(42)

In [ ]:
# Qui diciamo al codice dove sta il file che contiene i dati (è un file csv)
# glielo facciamo leggere e togliamo alcuni dati che non ci interessano (con df.drop)
df = pd.read_csv("../data/LifeExpectancyData.csv")
df = df.drop_duplicates(subset='Country')
df.set_index('Country', inplace=True, drop=True)
df.drop('Year', inplace=True, axis=1)
df.drop(' thinness  1-19 years', inplace=True, axis=1)
df.drop('infant deaths', inplace=True, axis=1)
df.drop('Income composition of resources', inplace=True, axis=1)
df.drop('percentage expenditure', inplace=True, axis=1)
df.drop('Adult Mortality', inplace=True, axis=1)
df=df.fillna(df.mean(numeric_only=True))
str_columns = df.select_dtypes(include=['object'])
df = pd.get_dummies(df, columns=str_columns.columns)

# Vediamo i dati!
Abbiamo immagazzinato i nostri dati in una tabella (nello specifico, in un *dataframe*, per questo abbiamo chiamato la tabella "df").

Le righe contengono i paesi, e le colonne delle caratteristiche (*features*) per ogni paese. 

Esegui la prossima cella per visualizzare i dati!

In [ ]:
df

# Predire l'aspettativa di vita nei paesi del mondo


Ci interessa capire quali fattori influenzano l'aspettativa di vita in un Paese.

Possiamo immaginare che le caratteristiche nelle colonne qui sopra contribuiscano positivamente o negativamente all'aspettativa di vita, ma non sappiamo in che modo: questo è quello che vogliamo *imparare dai dati*. Lo faremo attraverso quello che chiamiamo un *modello*, che ha lo scopo di "predire" la quantità che studiamo (l'aspettativa di vita) partendo dalle altre caratteristiche.

Se nella tabella abbiamo tutte le caratteristiche rilevanti, più preciso sarà il nostro modello, più accuratamente potremo predire l'aspettativa di vita in un paese usando i dati presenti nella tabella.

Per prima cosa, visto che predire l'aspettativa di vita usando come dato proprio l'aspettativa di vita sarebbe barare, la rimuoviamo dalle colonne della tabella e la mettiamo in una lista a parte, che chiamiamo y. Questa y ci serve sia per istruire il modello, sia poi per valutare quanto questo sarà bravo nelle sue previsioni. Premi play qui sotto.

In [ ]:
# Definiamo y
y=df['Life expectancy ']
# Eliminiamo l'aspettativa di vita dalla tabella precedente
df.drop('Life expectancy ', inplace=True, axis=1)
# Invece X sarà il resto della tabella
X=df.copy()

Abbiamo 193 stati.
Se usassimo tutti gli stati per imparare la previsione che riteniamo più accurata, non avremmo nessun modo per sapere se il nostro modello funzioni anche su stati diversi o, ad esempio, sugli stessi stati ma tra 10 anni.
Dobbiamo quindi dividere questo insieme di dati in due gruppi, uno per "allenare" il modello (*set di training*) e l'altro per testare se sta funzionando bene e quanto bene funziona (*set di test*).

In [ ]:
# Di solito si usano circa 3/4 dei dati a disposizione per allenare il modello
# abbiamo i dati per 193 stati -> i suoi tre quarti son circa 140
training_samples=140
# MODIFICABILE: per ora esegui la cella lasciando così il numero di paesi usati per allenare
# alla fine puoi tornare qui e decidere di cambiarlo per vedere cosa succede
# Se lo cambi, dovrai poi eseguire di nuovo anche tutte le celle più sotto
# per vedere l'impatto che ha: non le ricalcola in automatico!

# Pre-processing
La prossima cella nascosta fa un po' di lavoro sporco: divide il dataset in set di training (allenamento) e set di test (verifica) mescolando gli stati casualmente, e "riscala" le variabili in modo che i loro valori abbiano ampiezze comparabili. Queste operazioni fanno parte di un processo che si chiama "pre-processing" e che è molto utile per ottenere buoni risultati. Se chiedete in giro, però, scoprirete che anche questo non è molto divertente...

Se vuoi saperne di più, questo è il momento giusto per una domanda!

In [ ]:
# Qui facciamo delle divisioni in modo che tutti i valori stiano tra 0 e 1.
# Questo aiuterà poi per confrontare quale aspetto impatta di più sull'aspettativa di vita
X=X.astype(float)  
X=((X - X.min()) / (X.max() - X.min()))

indexes = list(range(len(y)))
random.shuffle(indexes)

X=X.iloc[indexes]
y=y.iloc[indexes]

X_train=X[:training_samples]
X_test=X[training_samples:]
y_train=y[:training_samples]
y_test=y[training_samples:]

# Introduciamo e alleniamo il modello
Finalmente facciamo quanto ci è stato commissionato: cercare di predire l'aspettativa di vita media nei diversi paesi partendo dai valori nelle altre colonne della tabella, e capire quali di esse sono più importanti in questa previsione.

Il modello che usiamo oggi si chiama **Regressione Lineare** e vuole prevedere l'aspettativa di vita di un paese partendo dagli altri dati che conosciamo per quel paese, ad esempio PIL o gli anni di scuola che si fanno in quel paese.

Questo modello prevede di calcolare l'aspettativa di vita in un Paese moltiplicando i valori nelle colonne della tabella (che chiameremo $x_1$, $x_2$, ..., $x_n$ e sono, per esempio, il PIL o gli anni di istruzione) per dei numeri (che chiameremo *pesi* e indicheremo come $w_1$, $w_2$, $w_n$) e poi sommando tutto quanto. 

Di fatto è una media pesata:

$$y= w_1 * x_1 + w_2 * x_2 + ... + w_n * x_n$$

---

La media pesata è qualcosa che hai probabilmente già incontrato... Per esempio, immagina di dover fare un esame che certifichi il tuo livello di inglese. L'esame è composto da una parte di reading, una listening, una di writing e una di speaking, ma, per il voto finale, queste parti contano in modo diverso: il reading 40%, tutte le altre 20%. Per ottenere il tuo voto finale, quindi, non puoi fare semplicemente la media tra i 4 voti, perché il reading vale di più. Dovrai calcolare la media pesata così:

$$voto=0,4*read+0,2*listen+0,2*write+0,2*speak$$

dove read, listen, write e speak sono i voti di quelle 4 parti, mentre 0,4 e i vari 0,2 sono quelli che sopra abbiamo chiamato "pesi", perché indicano quanto è importante (quanto "pesa") il pezzo che stai sommando.

---

Ora torniamo al nostro caso dell'aspettativa di vita. Mentre per l'esame i "pesi" li conoscevamo in partenza, qui non abbiamo idea di quanto valgano e anzi, il punto è proprio quello di trovare quali pesi $w_i$ permettono di ottenere una somma (la $y$) più simile possibile all'aspettativa di vita misurata davvero, usando la formula scritta sopra:

$$y= w_1 * x_1 + w_2 * x_2 + ... + w_n * x_n$$

Ma attenzione, mentre i valori delle $x$ (il prodotto interno lordo, la presenza di HIV, ecc) e la y (l'aspettativa di vita) variano da Stato a Stato, i pesi che vogliamo trovare devono essere **gli stessi per tutti gli Stati**! Sarà quindi impossibile fissarli e ottenere per tutti gli Stati la previsione giusta e precisa... Dovremo trovare i numeri che "sbagliano meno"... Che ci danno una previsione "abbastanza buona". Il bello è che, se siamo abbastanza bravi, poi potremo usare questi stessi pesi per predire l'aspettativa di vita anche di altri paesi per cui abbiamo i dati, perché i pesi che troviamo saranno validi in generale!

Il computer sa fare questa "ottimizzazione" da solo, cioè trovare i pesi migliori, che prevedono delle aspettative di vita simili a quelle vere, usando i due comandi qui sotto: premi play!

In [ ]:
# Model è la "nostra" LinearRegression
model=LinearRegression();
# Con la funzione fit, il modello "impara" da solo i pesi ottimali
model.fit(X_train,y_train);

Abbiamo detto che il nostro modello non può predire le aspettative in modo preciso, perché obblighiamo i "pesi" a essere uguali per tutti i paesi quindi non è abbastanza flessibile. Ma come capire quanto le aspettative di vita calcolate si discostano dalla realtà?

Spesso, una buona pratica è quella di fare un grafico! Il nostro grafico avrà:
- sulla *x* il valore vero dell'aspettativa di vita, quello che ci viene dato nella tabella
- sulla *y* il valore che il nostro modello prevede

Se il nostro modello fosse sempre corretto, *x* e *y* sarebbero uguali, quindi il grafico visualizzato sarebbe *y=x*, la diagonale! 

In generale, più i punti sono vicini alla diagonale, più l'aspettativa vera e quella predetta si somigliano e quindi migliore è la predizione.

**Completa tu il codice, dove indicato nel commento, per visualizzare quanto è buono il nostro modello.**

In [ ]:
# Calcoliamo l'aspettativa predetta per gli stati usati nel train
y_train_pred=model.predict(X_train)

############# GRAFICO ############# 

# Questo grafico (sarà in azzurro in figura) rappresenta:
# - con dei pallini i dati relativi ai singoli Paesi, usati nel training
# - con una retta la previsione fatta dal nostro modello
sns.regplot(x=y_train, y=y_train_pred).set(title='Stati usati nel training')

# Qui vogliamo inserire il grafico (sarà in arancione in figura) che rappresenta è y=x, la diagonale
# Suggerimento: puoi usare
# - plt.plot
# - y_train, lista che raccoglie tutti i veri valori di aspettativa di vita

############# SCRIVI QUI IL TUO CODICE #############

ax = plt.gca()
ax.set_xlabel("Aspettativa di vita vera (da tabella)")
ax.set_ylabel("Aspettativa di vita predetta (dal modello)");

Non stanno proprio sulla diagonale, ma ci stanno attorno... è un buon risultato. Vorrà dire che non possiamo considerare la previsione molto precisa (sbaglia facilmente di 5-10 anni), però non è neanche schifosa... ci prende abbastanza. Considerate che è un modello molto semplice, quindi questo risultato è sensato.

# Testiamo il modello

Ora che il modello ha imparato i pesi, possiamo usarlo per predire le aspettative di vita per gli stati di test (quelli che non abbiamo usato per decidere il valore dei pesi), usando la funzione **predict**:

In [ ]:
y_pred=model.predict(X_test)

Controlliamo anche qui come sono le previsioni facendo lo stesso grafico, ma usando questa volta *solo gli stati che non erano stati usati per far imparare il modello*.

**Completa tu il codice, dove indicato nel commento, per visualizzare quanto è buono il nostro modello.**

In [ ]:
# Suggerimento: usa il comando sns.regplot come fatto nella precedente, 
# ma questa volta usa i dati di test (y_test e y_pred, calcolato sopra usando y_test)!

############# SCRIVI QUI IL TUO CODICE #############

plt.plot(y_test, y_test)
ax = plt.gca()
ax.set_xlabel("Aspettativa di vita vera (da tabella)")
ax.set_ylabel("Aspettativa di vita predetta (dal modello)");

La precisione (cioè quanto i punti stanno vicini alla diagonale) è simile a quella che c'è negli stati usati per il training e questo è un ottimo risultato!

Significa che possiamo usare questo modello per farci un'idea di cosa succederà anche in casi di cui non abbiamo la risposta (o per fare previsioni sull'impatto di decisioni come diminuire la spesa annuale per la sanità pubblica, o diminuire gli anni di scuola, o ridurre sensibilmente il numero di malati di HIV puntando sulla prevenzione ecc) e ... avere una certa fiducia che **in media** ci darà dei risultati sensati.

**Complimenti! Avete creato uno strumento che potrebbe essere molto utile nelle mani dei governanti dei paesi per capire su cosa puntare per migliorare l'aspettativa di vita dei propri abitanti!**

# Quali fattori contano di più?
Il valore dei pesi che il modello ha scelto ci aiuta acapire come e quanto ciascuna caratteristica (*feature*) $x_i$ contribuisca a determinare il valore dell'aspettativa di vita $y$ secondo questo modello.

Quali delle nostre caratteristiche (le $x_i$) hanno più peso nel modello per predire l'aspettativa di vita? Ci basta vedere quanto valgono i pesi $w_i$ che corrispondono a quelle $x_i$: più un peso è grande e positivo, più influisce positivamente nell'aspettativa di vita, più un peso è grande e negativo più influisce negativamente nell'aspettativa di vita, mentre se il peso è piccolo vuol dire che quella variabile non ha molto a che fare con l'aspettativa di vita.

Guardiamoli insieme: premi play!

In [ ]:
plt.bar(df.columns,model.coef_) # df.columns sono gli x_i (HIV/AIDS, istruzione, ...), model.coef_ sono i pesi w_i
plt.axhline(linestyle='--', c='k')
plt.xticks(rotation=90);
plt.ylabel("Valore del peso $w_i$");

# Quiz

Il grafico ci comunica che le variabili che più aumentano l'aspettativa di vita media sono la spesa totale per la sanità (nella tabella indicata con *total expenditure*) e il livello di istruzione medio. E ci dice anche che una vasta presenza di HIV/AIDS e una mortalità infantile morto forte abbassano l'aspettativa di vita media. Sono risultati che più o meno a pelle ci aspettavamo.

Tuttavia, c'è qualcosa di strano: com'è possibile che la presenza di morbillo, poliomelite e difterite ("Measles", "Polio" e "Diphteria" in inglese) aumenti l'aspettativa di vita di una nazione? Pensateci insieme, trovate un aiuto nella cella qui sotto. Quali potrebbero essere le cause di questo strano risultato?

In [ ]:
corr = df.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))  # You can adjust the size as needed
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm')

# Show the plot
plt.show()

Questo grafico, in maniera non troppo semplice, ci dà una spiegazione: ci spiega infatti quanto sono collegate tra loro le varie *features*. Più alto è il valore, quindi più rossa è una casella, più le due *features* associate a quella casella sono strettamente legate tra loro. Ovviamente, ogni *feature* è molto legata a se stessa, quindi la diagonale è molto rossa.

Una cosa interessante che possiamo notare, però, è che, osservando la riga della scolarizzazione ("Schooling"), questa *feature* è legata in maniera abbastanza forte alla presenza di poliomielite e difterite.

### Come mai queste strane relazioni?
Una spiegazione del perché la presenza di morbillo, poliomielite e difterite sembrano aumentare l'aspettativa di vita può essere legata al fatto stesso di avere a disposizione questi dati. Ovvero, se un Paese ha le risorse per monitorare e raccogliere dati accurati su queste malattie, è probabile che abbia anche un sistema sanitario ben sviluppato, che contribuisce a una maggiore aspettativa di vita!